In [1]:
import pandas as pd
import pymysql
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import matplotlib.font_manager as fm
import warnings
import random

warnings.filterwarnings("ignore")
plt.style.use ('ggplot')
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.size'] = 7.5
plt.rcParams['axes.unicode_minus'] = False

In [2]:
username = "*****"
password = "*****!"
hostname = "ec2-15-152-211-160.ap-northeast-3.compute.amazonaws.com"
database_name = "Data_Mart"

engine = create_engine(
    "mysql+pymysql://{user}:{pw}@{host}/{db}".format(
        user=username, pw=password, db=database_name, host=hostname
    )
)

credit_analysis_model_b = pd.read_sql("SELECT * FROM credit_analysis_model_b", con=engine)
engine.dispose()

In [3]:
# credit_analysis_model_b 테이블의 피쳐별 극단값 3개씩(0.001%) 제거

columns_to_process = credit_analysis_model_b.columns[4:]
processed_data = credit_analysis_model_b.iloc[:, :4].copy()

for column in columns_to_process:
    sorted_column = credit_analysis_model_b[column].sort_values()
    processed_column = sorted_column.iloc[3:-3]
    processed_data[column] = processed_column

processed_data.dropna(inplace=True)
credit_analysis_model_b = processed_data.copy()

In [4]:
# 산업별 테이블 생성
credit_analysis_model_b.drop(['corp','stock_code','rank'], axis = 1, inplace = True)
industry_average_investment = credit_analysis_model_b.groupby(['sector', 'year']).mean()

In [7]:
# 산업별 연도 전부 평균

industry_average_investment_year = industry_average_investment.reset_index().drop(['year'],axis=1).groupby(['sector']).mean()
industry_average_investment_year.reset_index(inplace = True)

In [8]:
industry_average_investment_year

,sector,ebitda_margin,ebitda_to_interest_expense,debt_ratio,dependence_on_net_borrowings,net_borrowings_to_ebitda,revenue,cogs,selling_general_administrative_expenses,ebit,...,enterprise_value_to_ebitda,operating_income,net_income,fixed_assets,total_asset_turnover,net_working_capital_turnover,fixed_asset_turnover,accounts_receivable_turnover,inventory_turnover,accounts_payable_turnover
0,건설업,6.464033,112.397425,0.642296,0.388109,0.177615,8.366209e+12,7.384990e+12,4.209603e+11,4.680458e+11,...,4.545132,5.260082e+11,3.358543e+11,1.008271e+12,0.773923,8.827771,11.269384,7.185198,7.149324,8.537141
1,기계,3.114180,1.957000,0.656298,0.450181,5.381187,4.771535e+12,3.995630e+12,4.950552e+11,1.103939e+11,...,22.273339,2.808493e+11,8.024469e+10,3.649377e+12,0.761411,29.088451,3.216073,7.443143,8.427614,7.646822
2,기타금융,3.969036,1.857222,0.592448,0.479645,-2.427306,1.409786e+13,1.177234e+13,1.584242e+12,5.541383e+11,...,0.767443,7.349605e+11,3.585851e+11,7.269488e+12,0.779618,3.734532,2.702507,8.539585,9.412730,8.133490
3,기타제조업,0.062649,-0.001280,0.649493,0.713231,-21.204685,1.218940e+11,9.948609e+10,1.602071e+10,1.070327e+08,...,-21.669931,6.387206e+09,2.411764e+09,2.758941e+11,0.322105,-2.487064,0.462083,4.570977,2.349213,6.201171
4,"농업, 임업 및 어업",6.375798,3.858204,0.523605,0.305139,0.395282,6.781653e+11,6.030815e+11,3.531118e+10,4.161466e+10,...,1.742259,3.977256e+10,3.486837e+10,4.468886e+11,0.596039,-11.797105,1.513946,7.674090,5.191921,3.806120
5,비금속광물,10.489184,8.651072,0.417039,0.234172,1.677919,1.338413e+12,1.048550e+12,1.503950e+11,1.338380e+11,...,9.604941,1.394675e+11,1.314394e+11,1.730168e+12,0.498195,3.269280,0.843893,4.576340,8.044944,8.472863
6,서비스업,7.913824,11.485909,0.533911,2.099351,8.292210,2.037045e+12,1.266297e+12,2.477478e+11,1.923358e+11,...,23.474704,1.119933e+12,1.358014e+11,1.013566e+12,0.695280,2.992015,3.369547,20.934814,906.349552,44.759892
7,섬유의복,2.689106,5.128537,0.467650,0.507340,12.681569,1.151881e+12,7.367291e+11,3.528350e+11,5.860575e+10,...,52.636485,6.214824e+10,3.627107e+10,2.992457e+11,1.230650,6.474380,5.249501,10.489564,3.390826,15.273932
8,운수장비,-1.905732,1.143294,0.629779,0.662531,15.954310,1.399511e+13,1.179770e+13,1.364790e+12,6.270898e+11,...,37.471764,5.593618e+11,4.666528e+11,4.794716e+12,0.978622,-0.118377,2.647001,8.907846,10.074825,6.200486
9,운수창고업,-1.746776,1.955286,0.699503,0.361602,0.886717,8.420281e+12,7.503489e+12,4.671849e+11,2.152171e+11,...,6.109671,4.496064e+11,1.542990e+11,6.546819e+12,0.791073,-13.488678,1.670256,10.289470,120.842199,14.056827


In [9]:
username = "multi"
password = "Campus123!"
hostname = "ec2-15-152-211-160.ap-northeast-3.compute.amazonaws.com"
database_name = "Data_Mart"

desired_table_name = "industry_average_investment_year"

cnx = pymysql.connect(user=username, password=password, host=hostname)
cursor = cnx.cursor()

engine = create_engine(
    "mysql+pymysql://{user}:{pw}@{host}/{db}".format(
        user=username, pw=password, db=database_name, host=hostname
    )
)
Session = sessionmaker(bind=engine)
session = Session()

try:
    industry_average_investment_year.to_sql(
        desired_table_name, con=engine, if_exists="replace", index=False, chunksize=1000
    )
    session.commit()
except:
    session.rollback()
    raise
finally:
    session.close()

cursor.close()
cnx.close()